In [1]:
import sys,os
sys.path.append(os.curdir+"/deep-learning-from-scratch")
from dataset.mnist import load_mnist

import numpy as np
from PIL import Image
import matplotlib.pylab as plt

In [27]:
def debug(variable):
    print(variable, '=', repr(eval(variable)))

In [9]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
    
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        return out
    
    def backward(self, dout):
        # 積算の逆伝搬は 
        # f(x,y) = x*y
        # df/dx = 1 * y
        # df/dy = 1 * x
        dx = self.y * dout
        dy = self.x * dout
        return dx, dy

In [16]:
apple = 100
apple_num = 2
tax = 1.1

# layer
apple_layer = MulLayer()
tax_layer = MulLayer()

# forward
apple_price = apple_layer.forward(apple, apple_num)
sum = int(tax_layer.forward(apple_price, tax))
print(sum)

# backward
dprice = 1
dapple_price, dtax = tax_layer.backward(dprice)
print(dapple_price)
print(dtax)

dapple, dnum = apple_layer.backward(dapple_price)
print(dapple)
print(dnum)

220
1.1
200
2.2
110.00000000000001


In [29]:
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x + y
        return out
    
    def backward(self, dout):
        # 加算の逆伝搬は 
        # f(x) = x + y
        # df/dx = 1
        # df/dy = 1
        dx = 1 * dout
        dy = 1 * dout
        return dx, dy        

In [30]:
# 順伝搬
apple = 100
apple_num = 2
apple_layer = MulLayer()
apple_price = apple_layer.forward(apple, apple_num)

orange = 150
orange_num = 3
orange_layer = MulLayer()
orange_price = orange_layer.forward(orange, orange_num)

sum_layer = AddLayer()
sum_price = sum_layer.forward(apple_price, orange_price)

tax = 1.1
tax_layer = MulLayer()
tax_price = tax_layer.forward(sum_price, tax)
print(tax_price)

715.0000000000001


In [37]:
# 逆伝搬
dout = 1
dsum_price, dtax_price = tax_layer.backward(dout)
debug("dsum_price")
debug("dtax_price")

dapple_price, dorange_price = sum_layer.backward(dsum_price)
debug("dapple_price")
debug("dorange_price")

dapple, dapple_num = apple_layer.backward(dapple_price)
debug("dapple")
debug("dapple_num")

dorange, dorange_num = orange_layer.backward(dorange_price)
debug("dorange")
debug("dorange_num")

dsum_price = 1.1
dtax_price = 650
dapple_price = 1.1
dorange_price = 1.1
dapple = 2.2
dapple_num = 110.00000000000001
dorange = 3.3000000000000003
dorange_num = 165.0


In [43]:
class Relu:
    def __init__(self):
        self.mask = None
    
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy
        out[self.mask] = 0
        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx
    
x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)
mask = (x <= 0)
print(mask)
x[mask] = 0
debug("x")

[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]
x = array([[1., 0.],
       [0., 3.]])


In [44]:
class Sigmoid:
    def __init__(self):
        self.out = None
    
    def forward(self, x):
        self.out = 1 / (1 + np.exp(x*-1))    
        return self.out

    def backward(self, dout):
        return dout * self.out * (1.0 - self.out)

In [45]:
class Affine:
    def __init__(self, W, b):
        self.x = None
        self.W = W
        self.b = b
        self.dW = None
        self.db = None
    
    def forward(self, x):
        self.x = x
        out = np.dot(self.W, x) + self.b
        return out

    def backward(self, dout):
        self.db = np.sum(dout, axis=0)
        self.dW = np.dot(self.x.T, dout)

        dx = np.dot(dout, self.W.T)
        return dx